In [ ]:
q = """
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>
"""             
from IPython.core.display import display, HTML
display(HTML(q))

display(HTML("<style>.container { width:100% !important; }</style>"))

q2 = """<style>
div.prompt {display:none}
</style>"""
display(HTML(q2))

# Coronavirus disease (COVID-19) Pandemic

![](https://2hfybu1lrdue3x9wnu1dvw7s-wpengine.netdna-ssl.com/wp-content/uploads/2020/03/covid-19-image.png)

Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.

Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment.  Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness.

The best way to prevent and slow down transmission is be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face. 

The COVID-19 virus spreads primarily through droplets of saliva or discharge from the nose when an infected person coughs or sneezes, so it’s important that you also practice respiratory etiquette (for example, by coughing into a flexed elbow).

At this time, there are no specific vaccines or treatments for COVID-19. However, there are many ongoing clinical trials evaluating potential treatments. WHO will continue to provide updated information as soon as clinical findings become available.

(source : [Coronaviras disease 2019 - World Health Organization](https://www.who.int/health-topics/coronavirus#tab=tab_1))

#### Prevention

To prevent infection and to slow transmission of COVID-19, do the following:

Wash your hands regularly with soap and water, or clean them with alcohol-based hand rub.
Maintain at least 1 metre distance between you and people coughing or sneezing.
Avoid touching your face.
Cover your mouth and nose when coughing or sneezing.
Stay home if you feel unwell.
Refrain from smoking and other activities that weaken the lungs.
Practice physical distancing by avoiding unnecessary travel and staying away from large groups of people.

#### Symptoms

COVID-19 affects different people in different ways. Most infected people will develop mild to moderate illness and recover without hospitalization.

**Most common symptoms:**

* fever.
* dry cough.
* tiredness.

**Serious symptoms:**

* difficulty breathing or shortness of breath.
* chest pain or pressure.
* loss of speech or movement.
* Seek immediate medical attention if you have serious symptoms.  Always call before visiting your doctor or health facility. 
* 
People with mild symptoms who are otherwise healthy should manage their symptoms at home. 

On average it takes 5–6 days from when someone is infected with the virus for symptoms to show, however it can take up to 14 days. 

In [ ]:
import pandas as pd
import os
import random

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.integrate import odeint

from plotly.offline import iplot, init_notebook_mode
import math
import bokeh 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from urllib.request import urlopen
import json
from dateutil import parser
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import row, column, gridplot
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span
import warnings
warnings.filterwarnings("ignore")
output_notebook(resources=INLINE)

# Index

1. [Covid-19 Global Situation](#1)
2. [India Story (Exploratory Data Analysis)](#2)
3. [Forecasting Spread in India](#3)
4. [Research for cure](#4)
5. [How we can contribute to this fight?](#5)

<a id='1'> </a>
          
# I. Covid-19 Global Situation

The coronavirus COVID-19 is affecting 212+ countries and territories around the world and 2 international conveyances. The virus is thought to be natural and has an animal origin,through spillover infection. Although, there are several theories about where the very first case (the so-called patient zero) may have originated. The first known case may trace back to 1 December 2019 in Wuhan, Hubei, China. Within a month, the number of coronavirus cases in Hubei gradually increased. By December 2019, the spread of infection was almost entirely driven by human-to-human transmission. On January 30, the World Health Organization declared the coronavirus outbreak a Global Public Health Emergency. It's spreading heavily all around the world.

Let's look into the spread of this COVID-19 disease over the period of time.

In [ ]:
covid_19_globally=pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

# Engineering global data
prev_index = 0
first_time = False
tmp = 0

# Adding historic counts of china
for i, row in covid_19_globally.iterrows():

    if(covid_19_globally.loc[i,'SNo'] < 1342 and covid_19_globally.loc[i,'Province/State']=='Hubei'):
        if(first_time):
            tmp = covid_19_globally.loc[i,'Confirmed']
            prev_index = i
            covid_19_globally.loc[i,'Confirmed'] = covid_19_globally.loc[i,'Confirmed'] + 593
            first_time = False
        else:
            increment = covid_19_globally.loc[i,'Confirmed'] - tmp
            tmp = covid_19_globally.loc[i,'Confirmed']
            covid_19_globally.loc[i,'Confirmed'] = covid_19_globally.loc[prev_index,'Confirmed'] + increment + 593
            prev_index = i
    

covid_19_globally.rename(columns={'Country/Region': 'Country', 'ObservationDate': 'Date'}, inplace=True)
covid_19_globally = covid_19_globally.fillna('unknow')

# Renaming data to join country code
covid_19_globally['Country'] = covid_19_globally['Country'].str.replace('US','United States')
covid_19_globally['Country'] = covid_19_globally['Country'].str.replace('UK','United Kingdom') 
covid_19_globally['Country'] = covid_19_globally['Country'].str.replace('Mainland China','China')
covid_19_globally['Country'] = covid_19_globally['Country'].str.replace('South Korea','Korea, South')
covid_19_globally['Country'] = covid_19_globally['Country'].str.replace('North Korea','Korea, North')
covid_19_globally['Country'] = covid_19_globally['Country'].str.replace('Macau','China')
covid_19_globally['Country'] = covid_19_globally['Country'].str.replace('Ivory Coast','Cote d\'Ivoire')

# Adding country code
country_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
country_codes = country_codes.drop('GDP (BILLIONS)', 1)
country_codes.rename(columns={'COUNTRY': 'Country', 'CODE': 'Code'}, inplace=True)

covid_19_globally = pd.merge(covid_19_globally,country_codes,on=['Country'])

In [ ]:
covid_19_timeFrame=covid_19_globally.groupby(['Date','Country','Code']).agg({'Confirmed':'sum'}).reset_index()

choro_map=px.choropleth(covid_19_timeFrame, 
                    locations="Country", 
                    locationmode = "country names",
                    color="Confirmed", 
                    hover_name="Country", 
                    animation_frame="Date",
                    color_continuous_scale=px.colors.sequential.Reds
                   )

choro_map.update_layout(
    title_text = 'Spread of COVID-19 over time',
    title_x = 0.5,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
choro_map.show()

#### Trends of Top 5 covid-19 affected countries and India

In [ ]:
# Top 5 countries

top_countries = covid_19_timeFrame.groupby(['Code','Country'])['Confirmed'].sum().reset_index()
top_countries = top_countries.sort_values('Confirmed', ascending=False)
top_countries = top_countries[:5]
top_countries_codes = list(top_countries['Country'])

countries_data = covid_19_globally[covid_19_globally['Country'].isin(top_countries_codes)]
countries_day_data = countries_data.groupby(['Date','Code','Country'])['Confirmed','Deaths','Recovered'].sum().reset_index()

countries_data_ind = covid_19_globally[covid_19_globally['Country']=='India']
countries_day_data_ind = countries_data.groupby(['Date','Code','Country'])['Confirmed','Deaths','Recovered'].sum().reset_index()

def get_country_trend(country_code):
    tmp = countries_day_data.loc[countries_day_data['Country']==country_code]

    trend = []
    trend.append( list(tmp['Confirmed'])[0] - list(tmp['Deaths'])[0] - list(tmp['Recovered'])[0])

    for i in range(1,len(tmp)):
        trend.append( list(tmp['Confirmed'])[i] - list(tmp['Deaths'])[i] - list(tmp['Recovered'])[i])
        
    return tmp,trend

trends = dict()
country_data = dict()
for code in top_countries_codes:
    country_data[code],trends[code] = get_country_trend(code)
    
# India
trend_ind = []
trend_ind.append( list(countries_data_ind['Confirmed'])[0] - list(countries_data_ind['Deaths'])[0] 
                 - list(countries_data_ind['Recovered'])[0])

for i in range(1,len(countries_data_ind)):
    trend_ind.append( list(countries_data_ind['Confirmed'])[i] - list(countries_data_ind['Deaths'])[i] 
                     - list(countries_data_ind['Recovered'])[i])

The novel coronavirus has impacted China, Italy, the US, and Spain the most, India too is seeing rise in COVID-19 continuasly. China, Germany and Spin showing decrease in cases but in the US and India it growing heavily.

In [ ]:
exponential_line_x = []
exponential_line_y = []
for i in range(16):
    exponential_line_x.append(i)
    exponential_line_y.append(i)

p1 = figure(plot_width=700, plot_height=350, title="Trends of Top 5 covid-19 affected countries")
p1.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p1.yaxis.axis_label = 'New confirmed cases (Log scale)'
line_colors = ['#003f5c','#58508d','#bc5090','#ff6361','#ffa600']

# Top 5 countries
p1.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=1)
for c,color in zip(top_countries_codes,line_colors):
    p1.line(np.log(list(country_data[c]['Confirmed'])),np.log(trends[c]), color=color, 
            legend_label=c, line_width=3)
    p1.circle(np.log(list(country_data[c]['Confirmed'])[-1]), np.log(trends[c][-1]), fill_color="white", size=5)
p1.legend.location = "bottom_right"

# India
p2 = figure(plot_width=700, plot_height=350, title="Trends of India")
p2.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p2.yaxis.axis_label = 'New confirmed cases (Log scale)'

# Top 5 countries
p2.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=1)

p2.line(np.log(list(countries_data_ind['Confirmed'])),np.log(trend_ind), color='#114f5d', 
        legend_label='India', line_width=3)
p2.circle(np.log(list(countries_data_ind['Confirmed'])[-1]), np.log(trend_ind[-1]), fill_color="white", size=5)
p2.legend.location = "bottom_right"

# output_file("coronavirus.html", title="coronavirus.py")

# display both
gp = gridplot(children=[[p1], [p2]])
show(gp)

<a id='2'> </a>
# II. India Story

The spread of novel coronavirus in India has quickened in the past few days with new cases of coronavirus and COVID-19 deaths being reported every day. After the first confirmed positive case was reported on January 30 in Kerala, the epidemic has expanded its footprints in the country, affecting more than 40,000 people.

As we see ealier in trend graphs, while the novel coronavirus has impacted China, Italy, the US, and Spain the most, India too is seeing rise in COVID-19 cases continuasly. After making its presence in Kerala first, the novel coronavirus spread to other cities/regions, including Bengaluru,Pune, Delhi, Jaipur, Agra, Hyderabad, Jammu and Kashmir.


In [ ]:
# india data
india_covid_19 = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
age_details = pd.read_csv('../input/covid19-in-india/AgeGroupDetails.csv')
hospital_beds = pd.read_csv('../input/covid19-in-india/HospitalBedsIndia.csv')
individual_details = pd.read_csv('../input/covid19-in-india/IndividualDetails.csv')
ICMR_details = pd.read_csv('../input/covid19-in-india/ICMRTestingDetails.csv')
ICMR_labs = pd.read_csv('../input/covid19-in-india/ICMRTestingLabs.csv')
state_testing = pd.read_csv('../input/covid19-in-india/StatewiseTestingDetails.csv')

confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
latest_data = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/04-04-2020.csv')

In [ ]:
dates = list(confirmed_df.columns[4:])
dates = list(pd.to_datetime(dates))
dates_india = dates[8:]

total = confirmed_df.groupby('Country/Region').sum().reset_index()
deths = deaths_df.groupby('Country/Region').sum().reset_index()
recovered = recovered_df.groupby('Country/Region').sum().reset_index()

k = total[total['Country/Region']=='India'].loc[:,'1/30/20':]
india_confirmed = k.values.tolist()[0] 

k = deths[deths['Country/Region']=='India'].loc[:,'1/30/20':]
india_deaths = k.values.tolist()[0] 

k = recovered[recovered['Country/Region']=='India'].loc[:,'1/30/20':]
india_recovered = k.values.tolist()[0] 

plt.figure(figsize= (12,6))
plt.yticks(fontsize = 10)
plt.ylabel("Total cases", fontsize = 12)
plt.title("Spread Trend", fontsize = 15)
plt.gca().yaxis.tick_right()
plt.gca().yaxis.set_label_position("right")
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

ax1 = plt.plot_date(y= india_confirmed,x= dates_india,label = 'Confirmed',linestyle ='-',color = '#386fa4')
ax2 = plt.plot_date(y= india_recovered,x= dates_india,label = 'Recovered',linestyle ='-',color = '#2e856e')
ax3 = plt.plot_date(y= india_deaths,x= dates_india,label = 'Death',linestyle ='-',color = '#b30000')
plt.legend();

## Statewise cases

In [ ]:
india_covid_19['Date'] = pd.to_datetime(india_covid_19['Date'])
state_testing['Date'] = pd.to_datetime(state_testing['Date'])

state_details = pd.pivot_table(india_covid_19, values=['Confirmed','Deaths','Cured'], index='State/UnionTerritory', aggfunc='max')
state_details['% total cases'] = round(state_details['Confirmed'] *100 / (state_details['Confirmed'].sum()),2)
state_details = state_details[['% total cases','Confirmed','Deaths','Cured']]
state_details['Recovery Rate'] = round(state_details['Cured'] / state_details['Confirmed'],2)
state_details['Death Rate'] = round(state_details['Deaths'] /state_details['Confirmed'], 2)
state_details = state_details.sort_values(by='Confirmed', ascending= False)
state_details[:-1].style.background_gradient(cmap='Reds')

### Containment Zones

In [ ]:
def color_zone(val):
    if val == 'Red Zone':
        color = 'background-color:red' 
    elif val == 'Green Zone':
        color = 'background-color:green'
    elif val == 'Orange Zone':
        color = 'background-color:orange'
    else:
        color = ''
    return color

In [ ]:
zone=pd.read_csv('/kaggle/input/covid-19-india-zone-classification/lockdownindiawarningzones.csv')
zone = zone.drop('S.no',axis=1)
zone = zone.sort_values('State')
zone.style.applymap(color_zone)

#### Counts over the time


In [ ]:

import IPython
IPython.display.HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1977187" data-url="https://flo.uri.sh/visualisation/1977187/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

## Demographic Analysis

COVID-19 is causing serious health, social and economic challenges, several of which are directly related to demographic factors. Given that older persons have a weaker immune system and are likely to have underlying chronic illness, they are particularly vulnerable to viruses like SARS-CoV-2.

Demography can help us understand how this pandemic has spread and has had a disproportionate effect on certain age groups, as well as why its spread affects everyone. 

### Worst hit states in tree plot

Maharashtra remains to be leading state in the number of Covid-19 cases reported till April 4. Also, Gujarat and Delhi have shown significant rise in the cases from past few days. 

In [ ]:
india_covid_19.rename(columns={'State/UnionTerritory': 'State', 'Cured': 'Recovered', 'Confirmed': 'Confirmed'}, inplace=True)
statewise_cases = pd.DataFrame(india_covid_19.groupby(['State'])['Confirmed', 'Deaths', 'Recovered'].max().reset_index())
statewise_cases["Country"] = "India" # in order to have a single root node
fig = px.treemap(statewise_cases, path=['Country','State'], values='Confirmed',
                  color='Confirmed', hover_data=['State'],
                  color_continuous_scale='Rainbow')
fig.show()

### Age group distribution of covid-19 affected population

In [ ]:
# plot
fig = plt.figure(figsize=(16, 8))
splot=sns.barplot( y='TotalCases', x='AgeGroup', data= age_details, palette='hls')
for p in splot.patches:
    plt.annotate('{}'.format(int(p.get_height())), 
                (p.get_x()+p.get_width()/2,
                p.get_height()+0.1),
                ha='center')
plt.title('Age distribution of confirm cases')
plt.box(on=None)
plt.show()



### Gender distribution of covid-19 affected population

In [ ]:
labels = ['Missing', 'Male', 'Female']
sizes = []
sizes.append(individual_details['gender'].isnull().sum())
sizes.append(list(individual_details['gender'].value_counts())[0])
sizes.append(list(individual_details['gender'].value_counts())[1])

explode = (0.1, 0.1, 0.1)
outer_colors = ['#c2cf6f','#ff9999','#c2c2f0']

labels1 = ['Male', 'Female']
sizes1 = []
sizes1.append(list(individual_details['gender'].value_counts())[0])
sizes1.append(list(individual_details['gender'].value_counts())[1])

explode1 = (0.1, 0.1)
outer_colors1 = ['#ff9999','#c2c2f0']

plt.subplot(1,2,1)
plt.pie(sizes,labels=labels, autopct='%1.1f%%', startangle=90, frame=True, 
                radius=3, colors=outer_colors, explode=explode, textprops={'fontsize':15})
fig1 = plt.gcf()
fig1.set_size_inches(14,8)
plt.title("Gender distribution",fontsize=16)
plt.axis('equal')
plt.axis('off')
plt.tight_layout()


plt.subplot(1,2,2)
plt.pie(sizes1,labels=labels1, autopct='%1.1f%%', startangle=90, frame=True, 
                radius=2.8, colors=outer_colors1, explode=explode1, textprops={'fontsize':15})
fig1 = plt.gcf()
fig1.set_size_inches(14,8)
plt.title("Gender distribution",fontsize=16)
plt.axis('equal')
plt.axis('off')
plt.tight_layout()


fig = plt.gcf()


The impact of the pandemic and its mortality rate are different for men and women.Infection and mortality is higher in men. The higher mortality risk for men appears in their 50s, and begins to taper off only at 90. Men with age of 20s highly pron to asymptomatic carrier. Sex-based immunological differences, a lower prevalence of smoking in women, and men developing co-morbid conditions such as hypertension at a younger age than women could have contributed to the higher mortality in men. In India of those infected with COVID‑19, more then 65% are men. 

## Availabel Heath care resources

### Hospitals in Urben and Rural Areas

In [ ]:
plt.figure(figsize=(15,30))
plt.subplot(2,1,1)
hospital_beds=hospital_beds.sort_values('NumUrbanHospitals_NHP18', ascending= False)[1:]
sns.barplot(data=hospital_beds,y='State/UT',x='NumUrbanHospitals_NHP18',color=sns.color_palette('hls')[3])
plt.title('Urban Hospitals per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospital_beds.shape[0]):
    count = hospital_beds.iloc[i]['NumUrbanHospitals_NHP18']
    plt.text(count+10,i,count,ha='center',va='center')

plt.subplot(2,1,2)
hospital_beds=hospital_beds.sort_values('NumRuralHospitals_NHP18', ascending= False)
sns.barplot(data=hospital_beds,y='State/UT',x='NumRuralHospitals_NHP18',color=sns.color_palette('RdBu')[0])
plt.title('Rural Hospitals per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospital_beds.shape[0]):
    count = hospital_beds.iloc[i]['NumRuralHospitals_NHP18']
    plt.text(count+100,i,count,ha='center',va='center')


### Testing across States

India is constantly upgrading its testing capabilities to keep a check on the spread of coronavirus infections in the country. The number of tests conducted has grown almost four times since April 1.

#### ICMR Testing centers

In [ ]:
labs = pd.read_csv("../input/covid19-in-india/ICMRTestingLabs.csv")
fig = px.treemap(labs, path=['state','city'],
                  color='city', hover_data=['lab','address'],
                  color_continuous_scale='reds')
fig.show()

In [ ]:
testing=state_testing.groupby('State').sum().sort_values('TotalSamples').reset_index()
fig = px.bar(testing, 
             x="TotalSamples",
             y="State", 
             orientation='h',
             height=800,
             title='Statewise test sample collected')
fig.show()

The state of Maharashtra conducted the most number of tests to detect the coronavirus (COVID-19); also confirming the highest number of positive cases. On April 24, India’s test positivity rate stood at 4.76%, a low value by international standards (because the median test positivity rate was around 5.73%). 

### Hospital beds for covid patients across state 


In [ ]:
plt.figure(figsize=(15,30))
plt.subplot(2,1,1)
hospitalBeds=hospital_beds.sort_values('NumUrbanBeds_NHP18', ascending= False)
sns.barplot(data=hospitalBeds,y='State/UT',x='NumUrbanBeds_NHP18',color=sns.color_palette('hls')[5])
plt.title('Rural Beds per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospitalBeds.shape[0]):
    count = hospitalBeds.iloc[i]['NumUrbanBeds_NHP18']
    plt.text(count+1500,i,count,ha='center',va='center')

plt.subplot(2,1,2)
hospitalBeds=hospitalBeds.sort_values('NumRuralBeds_NHP18', ascending= False)
sns.barplot(data=hospitalBeds,y='State/UT',x='NumRuralBeds_NHP18',color=sns.color_palette('RdBu')[2])
plt.title('Rural Beds per states')
plt.xlabel('Count')
plt.ylabel('States')
for i in range(hospitalBeds.shape[0]):
    count = hospitalBeds.iloc[i]['NumRuralBeds_NHP18']
    plt.text(count+1500,i,count,ha='center',va='center')

plt.tight_layout()
plt.show()

### Personal Protective Equipment (PPE)
coming soon....

<a id='3'> </a>
# III. Forecasting Spread in India

### Mathematical modelling of infectious diseases

### 1. Basic SEIR model

![](https://www.idmod.org/docs/hiv/_images/SEIR-SEIRS.png)

The SEIR models the flows of people between four states: 
susceptible (S), 
exposed (E), 
infected (I), and 
resistant (R). 

Each of those variables represents the number of people in those groups. The parameters alpha and beta partially control how fast people move from being susceptible to exposed (beta), from exposed to infected (sigma), and from infected to resistant (gamma). This model has two additional parameters; one is the background mortality (mu) which is unaffected by disease-state, while the other is vaccination (nu). The vaccination moves people from the susceptible to resistant directly, without becoming exposed or infected.

The SEIR differs from the SIR model in the addition of a latency period. Individuals who are exposed (E) have had contact with an infected person, but are not themselves infectious.

![](http://www.public.asu.edu/~hnesse/classes/seireqn.png)

**Terminology:**

* Beta	The parameter controlling how often a susceptible-infected contact results in a new exposure.
* Gamma	The rate an infected recovers and moves into the resistant phase.
* Sigma	The rate at which an exposed person becomes infective.
* Mu	The natural mortality rate (this is unrelated to disease). This models a population of a constant size,
* Initial susceptible	The number of susceptible individuals at the beginning of the model run.
* Initial exposed	The number of exposed individuals at the beginning of the model run.
* Initial infected	The number of infected individuals at the beginning of the model run.
* Initial recovered	The number of recovered individuals at the beginning of the model run.
* Days	Controls how long the model will run.


Compartments are boxes (the “states”), like Susceptible, Exposed, etc.
Transitions from one compartment to another are represented by arrows and it is transist as

![](https://miro.medium.com/max/920/1*5caEXgP-6ZG7HkrPECKAEA.png)

As an example, let’s look at the transition from Susceptibles to Infected in our SIR equations, with beta=2, a total population of 100, 10 infected and 90 susceptible. The rate is 1, as the infections happen immediately; the population the transition applies to is 2 * 10 = 20 individuals, as the 10 infected each infect 2 people; the probability is 90%, as 90/100 people can still be infected. It corresponds to this intuitive notation:

![](https://miro.medium.com/max/1222/1*We4K9njCiEMowVuran4mWA.png)

And more generally,

![](https://miro.medium.com/max/1400/1*NNO-CdlCZORoPYtllW343A.png)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os
from tqdm.notebook import tqdm
from scipy.integrate import solve_ivp
import numpy
import datetime
from datetime import timedelta

In [ ]:
# Susceptible equation
def dS_dt(S, I, R_t, T_inf):
    return -(R_t / T_inf) * I * S

# Exposed equation
def dE_dt(S, E, I, R_t, T_inf, T_inc):
    return (R_t / T_inf) * I * S - (T_inc**-1) * E

# Infected equation
def dI_dt(I, E, T_inc, T_inf):
    return (T_inc**-1) * E - (T_inf**-1) * I

# Recovered/Remove/deceased equation
def dR_dt(I, T_inf):
    return (T_inf**-1) * I

def SEIR_model(t, y, R_t, T_inf, T_inc):
    
    if callable(R_t):
        reproduction = R_t(t)
    else:
        reproduction = R_t
        
    S, E, I, R = y
    
    S_out = dS_dt(S, I, reproduction, T_inf)
    E_out = dE_dt(S, E, I, reproduction, T_inf, T_inc)
    I_out = dI_dt(I, E, T_inc, T_inf)
    R_out = dR_dt(I, T_inf)
    
    return [S_out, E_out, I_out, R_out]

In [ ]:
train = pd.read_csv('../input/covid19-global-forecasting-week-4/train.csv')
test = pd.read_csv('../input/covid19-global-forecasting-week-4/test.csv')
train['Date_datetime'] = train['Date'].apply(lambda x: (datetime.datetime.strptime(x, '%Y-%m-%d')))

In [ ]:
pop_info = pd.read_csv('/kaggle/input/covid19-population-data/population_data.csv')
country_pop = pop_info.query('Type == "Country/Region"')
province_pop = pop_info.query('Type == "Province/State"')
country_lookup = dict(zip(country_pop['Name'], country_pop['Population']))
province_lookup = dict(zip(province_pop['Name'], province_pop['Population']))

In [ ]:
def plot_model_and_predict(data, pop, solution, title='SEIR model'):
    sus, exp, inf, rec = solution.y
    
    f = plt.figure(figsize=(16,5))
    ax = f.add_subplot(1,2,1)
    #ax.plot(sus, 'b', label='Susceptible');
    ax.plot(exp, 'y', label='Exposed');
    ax.plot(inf, 'r', label='Infected');
    ax.plot(rec, 'c', label='Recovered/deceased');
    plt.title(title)
    plt.xlabel("Days", fontsize=10);
    plt.ylabel("Fraction of population", fontsize=10);
    plt.legend(loc='best');
    
    ax2 = f.add_subplot(1,2,2)
    preds = np.clip((inf + rec) * pop ,0,np.inf)
    ax2.plot(range(len(data)),preds[:len(data)],label = 'Predict ConfirmedCases')
    ax2.plot(range(len(data)),data['ConfirmedCases'])
    plt.title('Model predict and data')
    plt.ylabel("Population", fontsize=10);
    plt.xlabel("Days", fontsize=10);
    plt.legend(loc='best');

In [ ]:
Country = 'India'
N = pop_info[pop_info['Name']==Country]['Population'].tolist()[0] # India Population 

# Load dataset of Hubei
train_loc = train[train['Country_Region']==Country].query('ConfirmedCases > 0')
if len(train_loc)==0:
    train_loc = train[train['Province_State']==Country].query('ConfirmedCases > 0')

n_infected = train_loc['ConfirmedCases'].iloc[0] # start from first comfirmedcase on dataset first date
max_days = len(train_loc)# how many days want to predict

# Initial stat for SEIR model
s = (N - n_infected)/ N
e = 0.
i = n_infected / N
r = 0.

# Define all variable of SEIR model 
T_inc = 5.2  # average incubation period
T_inf = 2.9 # average infectious period
R_0 = 3.954 # reproduction number

## Solve the SEIR model 
sol = solve_ivp(SEIR_model, [0, max_days], [s, e, i, r], args=(R_0, T_inf, T_inc), 
                t_eval=np.arange(max_days))

## Plot result
# plot_model_and_predict(train_loc, N, sol, title = 'SEIR Model (without intervention)')


In [ ]:
def cumsum_signal(vec):
    temp_val = 0
    vec_new = []
    for i in vec:
        if i > temp_val:
            vec_new.append(i)
            temp_val = i
        else:
            vec_new.append(temp_val)
    return vec_new


In [ ]:
# Use a constant reproduction number
def eval_model_const(params, data, population, return_solution=False, forecast_days=0):
    R_0, cfr = params # Paramaters, R0 and cfr 
    N = population # Population of each country
    n_infected = data['ConfirmedCases'].iloc[0] # start from first comfirmedcase on dataset first date
    max_days = len(data) + forecast_days # How many days want to predict
    s, e, i, r = (N - n_infected)/ N, 0, n_infected / N, 0 #Initial stat for SEIR model
    
    # R0 become half after intervention days
    def time_varying_reproduction(t):
        if t > 80: # we set intervention days = 80
            return R_0 * 0.5
        else:
            return R_0
    
    # Solve the SEIR differential equation.
    sol = solve_ivp(SEIR_model, [0, max_days], [s, e, i, r], args=(time_varying_reproduction, T_inf, T_inc),
                    t_eval=np.arange(0, max_days))
    
    sus, exp, inf, rec = sol.y
    # Predict confirmedcase
    y_pred_cases = np.clip((inf + rec) * N ,0,np.inf)
    y_true_cases = data['ConfirmedCases'].values
    
    # Predict Fatalities by remove * fatality rate(cfr)
    y_pred_fat = np.clip(rec*N* cfr, 0, np.inf)
    y_true_fat = data['Fatalities'].values
    
    optim_days = min(20, len(data))  # Days to optimise for
    weights = 1 / np.arange(1, optim_days+1)[::-1]  # Recent data is more heavily weighted
    
    # using mean squre log error to evaluate
    msle_cases = mean_squared_log_error(y_true_cases[-optim_days:], y_pred_cases[-optim_days:], weights)
    msle_fat = mean_squared_log_error(y_true_fat[-optim_days:], y_pred_fat[-optim_days:], weights)
    msle_final = np.mean([msle_cases, msle_fat])
    
    if return_solution:
        return msle_final, sol
    else:
        return msle_final


In [ ]:
# Use a Hill decayed reproduction number
def eval_model_decay(params, data, population, return_solution=False, forecast_days=0):
    R_0, cfr, k, L = params # Paramaters, R0 and cfr 
    N = population # Population of each country
    n_infected = data['ConfirmedCases'].iloc[0] # start from first comfirmedcase on dataset first date
    max_days = len(data) + forecast_days # How many days want to predict
    s, e, i, r = (N - n_infected)/ N, 0, n_infected / N, 0 #Initial stat for SEIR model
    
    # https://github.com/SwissTPH/openmalaria/wiki/ModelDecayFunctions   
    # Hill decay. Initial values: R_0=2.2, k=2, L=50
    def time_varying_reproduction(t): 
        return R_0 / (1 + (t/L)**k)
    
    # Solve the SEIR differential equation.
    sol = solve_ivp(SEIR_model, [0, max_days], [s, e, i, r], args=(time_varying_reproduction, T_inf, T_inc),
                    t_eval=np.arange(0, max_days))
    
    sus, exp, inf, rec = sol.y
    # Predict confirmedcase
    y_pred_cases = np.clip((inf + rec) * N ,0,np.inf)
    y_true_cases = data['ConfirmedCases'].values
    
    # Predict Fatalities by remove * fatality rate(cfr)
    y_pred_fat = np.clip(rec*N* cfr, 0, np.inf)
    y_true_fat = data['Fatalities'].values
    
    optim_days = min(20, len(data))  # Days to optimise for
    weights = 1 / np.arange(1, optim_days+1)[::-1]  # Recent data is more heavily weighted
    
    # using mean squre log error to evaluate
    msle_cases = mean_squared_log_error(y_true_cases[-optim_days:], y_pred_cases[-optim_days:], weights)
    msle_fat = mean_squared_log_error(y_true_fat[-optim_days:], y_pred_fat[-optim_days:], weights)
    msle_final = np.mean([msle_cases, msle_fat])
    
    if return_solution:
        return msle_final, sol
    else:
        return msle_final



In [ ]:
import plotly.express as px
from matplotlib import dates
import plotly.graph_objects as go

def fit_model_new(data, area_name, initial_guess=[2.2, 0.02, 2, 50], 
              bounds=((1, 20), (0, 0.15), (1, 3), (1, 100)), make_plot=True, decay_mode = None):
    
    if area_name in ['France']:# France last data looks weird, remove it
        train = data.query('ConfirmedCases > 0').copy()[:-1]
    #elif area_name in ['Virgin Islands']:
    #    train = data[:-3].query('ConfirmedCases > 0').copy()
    else:
        train = data.query('ConfirmedCases > 0').copy()
    
    ####### Split Train & Valid #######
    #valid_data = train[-1:]
    train_data = train
    
    ####### If this country have no ConfirmedCase, return 0 #######
    if len(train_data) == 0:
        result_zero = np.zeros((43))
        return pd.DataFrame({'ConfirmedCases':result_zero,'Fatalities':result_zero}), 0 
    
    ####### Load the population of area #######
    try:
        #population = province_lookup[area_name]
        population = pop_info[pop_info['Name']==area_name]['Population'].tolist()[0]
    except IndexError:
        print ('country not in population set, '+str(area_name))
        population = 1000000 
    
    
    if area_name == 'US':
        population = 327200000
    if area_name == 'Global':
        population = 7744240900
        
    cases_per_million = train_data['ConfirmedCases'].max() * 10**6 / population
    n_infected = train_data['ConfirmedCases'].iloc[0]
    
    ####### Total case/popuplation below 1, reduce country population #######
    if cases_per_million < 1:
        #print ('reduce pop divide by 100')
        population = population/100
        
    ####### Fit the real data by minimize the MSLE #######
    res_const = minimize(eval_model_const, [2.2, 0.02], bounds=((1, 20), (0, 0.15)),
                         args=(train_data, population, False),
                         method='L-BFGS-B')

    res_decay = minimize(eval_model_decay, initial_guess, bounds=bounds,
                         args=(train_data, population, False),
                         method='L-BFGS-B')
    
    ####### Align the date information #######
    test_end = datetime.datetime.strptime('2020-05-15','%Y-%m-%d')
    test_start = datetime.datetime.strptime('2020-03-26','%Y-%m-%d')
    train_test = data[data.Date_datetime>=test_start]
    test_period = (test_end - test_start).days
    train_max = train_data.Date_datetime.max()
    train_min = train_data.Date_datetime.min()
    add_date = 0
    delta_days =(test_end - train_max).days
    train_add_time=[]

    if train_min > test_start:
        add_date = (train_min-test_start).days
        last = train_min-timedelta(add_date)
        train_add_time = np.arange(last, train_min, dtype='datetime64[D]').tolist()
        train_add_time = pd.to_datetime(train_add_time)
        dates_all = train_add_time.append(pd.to_datetime(np.arange(train_min, test_end+timedelta(1), dtype='datetime64[D]')))
    else:
        dates_all = pd.to_datetime(np.arange(train_min, test_end+timedelta(1), dtype='datetime64[D]'))


    ####### Auto find the best decay function ####### 
    if decay_mode is None:
        if res_const.fun < res_decay.fun :
            msle, sol = eval_model_const(res_const.x, train_data, population, True, delta_days+add_date)
            res = res_const

        else:
            msle, sol = eval_model_decay(res_decay.x, train_data, population, True, delta_days+add_date)
            res = res_decay
            R_0, cfr, k, L = res.x
    else:
        if decay_mode =='day_decay':
            msle, sol = eval_model_const(res_const.x, train_data, population, True, delta_days+add_date)
            res = res_const
        else:
            msle, sol = eval_model_decay(res_decay.x, train_data, population, True, delta_days+add_date)
            res = res_decay
            R_0, cfr, k, L = res.x

    ####### Predict the result by using best fit paramater of SEIR model ####### 
    sus, exp, inf, rec = sol.y
    
    y_pred = pd.DataFrame({
        'ConfirmedCases': cumsum_signal(np.diff((inf + rec) * population, prepend=n_infected).cumsum()),
       # 'ConfirmedCases': [inf[0]*population for i in range(add_date)]+(np.clip((inf + rec) * population,0,np.inf)).tolist(),
       # 'Fatalities': [rec[0]*population for i in range(add_date)]+(np.clip(rec, 0, np.inf) * population * res.x[1]).tolist()
        'Fatalities': cumsum_signal((np.clip(rec * population * res.x[1], 0, np.inf)).tolist())
    })

    #y_pred_valid = y_pred.iloc[len(train_data):len(train_data)+len(valid_data)]
    y_pred_valid = y_pred.iloc[:len(train_data)]
    y_pred_test = pd.concat([train_test[['ConfirmedCases', 'Fatalities']],y_pred.iloc[-(delta_days):]], ignore_index=True)
    y_true_valid = train_data[['ConfirmedCases', 'Fatalities']]
    #y_true_valid = valid_data[['ConfirmedCases', 'Fatalities']]
    #print (len(y_pred),train_min)
    
    ####### Calculate MSLE ####### 
    valid_msle_cases = mean_squared_log_error(y_true_valid['ConfirmedCases'], y_pred_valid['ConfirmedCases'])
    valid_msle_fat = mean_squared_log_error(y_true_valid['Fatalities'], y_pred_valid['Fatalities'])
    valid_msle = np.mean([valid_msle_cases, valid_msle_fat])
    
    ####### Plot the fit result of train data and forecast after 300 days ####### 
    if make_plot:
        if len(res.x)<=2:
            print(f'Validation MSLE: {valid_msle:0.5f}, using intervention days decay, Reproduction number(R0) : {res.x[0]:0.5f}, Fatal rate : {res.x[1]:0.5f}')
        else:
            print(f'Validation MSLE: {valid_msle:0.5f}, using Hill decay, Reproduction number(R0) : {res.x[0]:0.5f}, Fatal rate : {res.x[1]:0.5f}, K : {res.x[2]:0.5f}, L: {res.x[3]:0.5f}')
        
        ####### Plot the fit result of train data dna SEIR model trends #######

        f = plt.figure(figsize=(16,5))
        ax = f.add_subplot(1,2,1)
        ax.plot(exp, 'y', label='Exposed');
        ax.plot(inf, 'r', label='Infected');
        ax.plot(rec, 'c', label='Recovered/deceased');
        plt.title('SEIR Model Trends')
        plt.xlabel("Days", fontsize=10);
        plt.ylabel("Fraction of population", fontsize=10);
        plt.legend(loc='best');
        #train_date_remove_year = train_data['Date_datetime'].apply(lambda date:'{:%m-%d}'.format(date))
        ax2 = f.add_subplot(1,2,2)
        xaxis = train_data['Date_datetime'].tolist()
        xaxis = dates.date2num(xaxis)
        hfmt = dates.DateFormatter('%m\n%d')
        ax2.xaxis.set_major_formatter(hfmt)
        ax2.plot(np.array(train_data['Date_datetime'], dtype='datetime64[D]'),train_data['ConfirmedCases'],label='Confirmed Cases (train)', c='g')
        ax2.plot(np.array(train_data['Date_datetime'], dtype='datetime64[D]'), y_pred['ConfirmedCases'][:len(train_data)],label='Cumulative modeled infections', c='r')
        #ax2.plot(np.array(valid_data['Date_datetime'], dtype='datetime64[D]'), y_true_valid['ConfirmedCases'],label='Confirmed Cases (valid)', c='b')
        #ax2.plot(np.array(valid_data['Date_datetime'], dtype='datetime64[D]'),y_pred_valid['ConfirmedCases'],label='Cumulative modeled infections (valid)', c='y')
        plt.title('Real ConfirmedCase and Predict ConfirmedCase')
        plt.legend(loc='best');
        plt.show()
            
        ####### Forecast 300 days after by using the best paramater of train data #######
        if len(res.x)>2:
            msle, sol = eval_model_decay(res.x, train_data, population, True, 300)
        else:
            msle, sol = eval_model_const(res.x, train_data, population, True, 300)
        
        sus, exp, inf, rec = sol.y
        
        y_pred = pd.DataFrame({
            'ConfirmedCases': cumsum_signal(np.diff((inf + rec) * population, prepend=n_infected).cumsum()),
            'Fatalities': cumsum_signal(np.clip(rec, 0, np.inf) * population * res.x[1])
        })
        
        ####### Plot 300 days after of each country #######
        start = train_min
        end = start + timedelta(len(y_pred))
        time_array = np.arange(start, end, dtype='datetime64[D]')

        max_day = numpy.where(inf == numpy.amax(inf))[0][0]
        where_time = time_array[max_day]
        pred_max_day = y_pred['ConfirmedCases'][max_day]
        xy_show_max_estimation = (where_time, max_day)
        
        con = y_pred['ConfirmedCases']
        fat = y_pred['Fatalities']
        max_day_con = numpy.where(con == numpy.amax(con))[0][0] # Find the max confimed case of each country
        max_day_fat = numpy.where(fat == numpy.amax(fat))[0][0]
        max_con = numpy.amax(con)
        max_fat = numpy.amax(fat)
        where_time_con = time_array[len(time_array)-50]
        xy_show_max_estimation_confirmed = (where_time_con, max_con)
        
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=time_array, y=y_pred['ConfirmedCases'].astype(int),
                            mode='lines',
                            line = dict(color='#b30000'),
                            name='Estimation Confirmed Case Start from '+ str(start.date())+ ' to ' +str(end.date())))
        fig.add_trace(go.Scatter(x=time_array, y=y_pred['Fatalities'].astype(int),
                            mode='lines',
                            line = dict(color='#FFE338'),
                            name='Estimation Fatalities Start from '+ str(start.date())+ ' to ' +str(end.date())))
        fig.add_trace(go.Scatter(x=time_array[:len(train)], y=train['ConfirmedCases'],
                            mode='lines',
                            name='Confirmed case until '+ str(train_max.date()),line = dict(color='#2e856e', width=4)))
        fig.add_trace(go.Scatter(x=time_array[:len(train)], y=train['Fatalities'],
                            mode='lines',
                            name='Fatalities case until '+ str(train_max.date()),line = dict(color='#386fa4', width=4)))
        fig.add_annotation(
            x=where_time_con,
            y=max_con-(max_con/30),
            showarrow=False,
            text="Estimate Max Case around:" +str(int(max_con)),
            font=dict(
                color="Blue",
                size=15
            ))
        fig.add_annotation(
            x=where_time_con,
            y=max_fat-(max_fat/30),
            showarrow=False,
            text="Estimate Max death around:" +str(int(max_fat)),
            font=dict(
                color="Blue",
                size=15
            ))
        fig.add_annotation(
            x=time_array[len(train)-1],
            y=train['ConfirmedCases'].tolist()[-1],
            showarrow=True,
            text=f"Real ConfirmedCase: " +str(int(train['ConfirmedCases'].tolist()[-1]))) 
        
        fig.add_annotation(
            x=time_array[len(train)-1],
            y=train['Fatalities'].tolist()[-1],
            showarrow=True,
            text=f"Real Fatalities: " +str(int(train['Fatalities'].tolist()[-1]))) 
        
        fig.add_annotation(
            x=where_time,
            y=pred_max_day,
            text='Infect start decrease from: ' + str(where_time))   
        
        fig.update_layout(title='Estimate Confirmed Case ,'+area_name+' Total population ='+ str(int(population)), legend_orientation="h",
                         yaxis = dict(showgrid = False), xaxis = dict(showgrid = False), plot_bgcolor='white')
        fig.show()

    return y_pred_test, valid_msle

In [ ]:
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_log_error, mean_squared_error

country = 'India'
if country not in train['Country_Region'].unique():
    country_pd_train = train[train['Province_State']==country]
else:
    country_pd_train = train[train['Country_Region']==country]

a,b = fit_model_new(country_pd_train,country,make_plot=True)

### 2. SEIR with deth and time compartment 

Coming soon .....

## Machine Learning Forecasting

Coming soon .....

<a id='4'> </a>
# IV. Research for cure

No medication or vaccine is approved to treat the disease. International research on vaccines and medicines in COVID‑19 is underway by government organisations, academic groups, and industry researchers. In March, the World Health Organisation initiated the "SOLIDARITY Trial" to assess the treatment effects of four existing antiviral compounds with the most promise of efficacy.

There has been a great deal of COVID-19 research, involving accelerated research processes and publishing shortcuts to meet the global demand. To minimise the impact of misinformation, medical professionals and the public are advised to expect rapid changes to available information, and to be attentive to retractions and other updates.

### Vaccine

Main article: [COVID-19 vaccine](https://en.wikipedia.org/wiki/COVID-19_vaccine)

There is no available vaccine, but various agencies are actively developing vaccine candidates. Previous work on SARS-CoV is being used because both SARS-CoV and SARS-CoV-2 use the ACE2 receptor to enter human cells. Three vaccination strategies are being investigated. First, researchers aim to build a whole virus vaccine. The use of such a virus, be it inactive or dead, aims to elicit a prompt immune response of the human body to a new infection with COVID‑19. A second strategy, subunit vaccines, aims to create a vaccine that sensitises the immune system to certain subunits of the virus. In the case of SARS-CoV-2, such research focuses on the S-spike protein that helps the virus intrude the ACE2 enzyme receptor. A third strategy is that of the nucleic acid vaccines (DNA or RNA vaccines, a novel technique for creating a vaccination). Experimental vaccines from any of these strategies would have to be tested for safety and efficacy.

On 16 March 2020, the first clinical trial of a vaccine started with four volunteers in Seattle, United States. The vaccine contains a harmless genetic code copied from the virus that causes the disease.

Antibody-dependent enhancement has been suggested as a potential challenge for vaccine development for SARS-COV-2, but this is controversial.

### Medications




Main article: [COVID-19 drug repurposing research](https://en.wikipedia.org/wiki/COVID-19_drug_repurposing_research)
At least 29 phase II–IV efficacy trials in COVID‑19 were concluded in March 2020 or scheduled to provide results in April from hospitals in China. There are more than 300 active clinical trials underway as of April 2020. Seven trials were evaluating already approved treatments, including four studies on hydroxychloroquine or chloroquine. Repurposed antiviral drugs make up most of the Chinese research, with nine phase III trials on remdesivir across several countries due to report by the end of April. Other candidates in trials include vasodilators, corticosteroids, immune therapies, lipoic acid, bevacizumab, and recombinant angiotensin-converting enzyme 2.

### Passive antibodies

Transferring purified and concentrated antibodies produced by the immune systems of those who have recovered from COVID‑19 to people who need them is being investigated as a non-vaccine method of passive immunisation. This strategy was tried for SARS with inconclusive results. Viral neutralisation is the anticipated mechanism of action by which passive antibody therapy can mediate defence against SARS-CoV-2. Other mechanisms, however, such as antibody-dependent cellular cytotoxicity and/or phagocytosis, may be possible. Other forms of passive antibody therapy, for example, using manufactured monoclonal antibodies, are in development. Production of convalescent serum, which consists of the liquid portion of the blood from recovered patients and contains antibodies specific to this virus, could be increased for quicker deployment.

#### In India

As the world races for coronavirus vaccine, plasma therapy is being tried as a temporary treatment to help patients recover. India too has started plasma therapy trials with hospitals in Maharashtra, Uttar Pradesh and Madhya Pradesh taking the lead. Other states and UTs are either awaiting approval from the ICMR r have got approval but have not yet started administering plsama yet. 


![](https://static.toiimg.com/photo/imgsize-436303,msid-75463201/75463201.jpg)

<a id='5'> </a>
# How we can contribute to this fight ? 

Preventive measures to reduce the chances of infection include staying at home, avoiding crowded places, keeping distance from others, washing hands with soap and water often and for at least 20 seconds, practising good respiratory hygiene, and avoiding touching the eyes, nose, or mouth with unwashed hands.

# Stay Home !

![](https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Covid-19-Transmission-graphic-01.gif/334px-Covid-19-Transmission-graphic-01.gif)

# Wash Hands Frequently !

![](https://www.paramounthealthcare.com/assets/images/news/covid-19_prevention.png)

Hope you find this notebook useful and interesting.
If you like this notebook, please upvote.***👍